In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from scipy.stats import skew,kurtosis
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from advanced_analysis_package import variable_treatment,analyze

In [2]:
from openpyxl import load_workbook,Workbook

In [3]:
os.chdir('../output_data/')

In [8]:
df = pd.read_csv('train_v01.csv',header=0)

In [9]:
df.columns

Index(['Date', 'Symbol', 'Series', 'Prev_Close', 'Open', 'High_pct_change',
       'Low_pct_change', 'Last_pct_change', 'Close', 'VWAP_pct_change',
       ...
       'Voltality_500', 'return_1d', 'return_2d', 'return_3d', 'return_4d',
       'return_5d', 'Voltality', 'Open_pct_change', 'Prev_Close_pct_change',
       'actual_return'],
      dtype='object', length=161)

In [10]:
os.chdir('../pickles')
numerical = pickle.load(open('numerical.pickle','rb'))
categorical = pickle.load(open('categorical.pickle','rb'))

In [11]:
os.chdir('../Statistics')

In [6]:
ignore_cols = ['Symbol','Series','Date','Prev_Close','Open','Close','Prev_Close_min','Prev_Close_max']

In [13]:
def automatic_outlier_treatment(data,s):
    
    lower_limit = data[s].quantile(q=.25)-1.5*(data[s].quantile(q=.75)-data[s].quantile(q=.25))
    upper_limit = data[s].quantile(q=.75)+1.5*(data[s].quantile(q=.75)-data[s].quantile(q=.25))

    frac_lower = float(np.unique(np.array(data.loc[(data[s].notnull()) & (data[s] < lower_limit),s]).shape[0]))/data.shape[0]
    frac_upper = float(np.unique(np.array(data.loc[(data[s].notnull()) & (data[s] > upper_limit),s]).shape[0]))/data.shape[0]

    p1,low_sigma3,low_sigma2 = data[s].quantile(q=.01),data[s].mean()-3*data[s].std(),data[s].mean()-2*data[s].std()
    p99,high_sigma3,high_sigma2 = data[s].quantile(q=.99),data[s].mean()+3*data[s].std(),data[s].mean()+2*data[s].std()
    p5,p95 = data[s].quantile(q=.05),data[s].quantile(q=.95)
    minima,maxima = data[s].min(),data[s].max()

    #deriving lower limit in case of aggresive treatment
    if low_sigma3 >= minima:
        b = min(p1,low_sigma3)
    elif low_sigma2>= minima:
        b = min(p1,low_sigma2)
    else:
        b=p1

    #deriving upper limit in case of aggresive treatment
    if high_sigma3 <= maxima:
        a = max(p99,high_sigma3)
    elif high_sigma2<= maxima:
        a = max(p99,high_sigma2)
    else:
        a=p99

    outlier_treatment = ''

    #lower end outlier treatment
    if frac_lower<=.05:
        variable_treatment.capping_and_flooring(data,s,a,b,high=False,low=True)
        if b==p1:
            outlier_treatment+='flooring at p1,'
        elif b==low_sigma2:
            outlier_treatment+='flooring at sigma2,'
        elif b==low_sigma3:
            outlier_treatment+='flooring at sigma3,'
    else:
        try:
            variable_treatment.exponential_smoothning(data,s,low=True,high=False)
            outlier_treatment+='lower line exponential smoothing,'
        except:
            pass

    #higher end outlier treatment
    if frac_upper<=.05:
        variable_treatment.capping_and_flooring(data,s,a,b,high=True,low=False)
        if b==p1:
            outlier_treatment+='capping at p99'
        elif b==low_sigma2:
            outlier_treatment+='capping at sigma2'
        elif b==low_sigma3:
            outlier_treatment+='Capping at sigma3'
    else:
        try:
            variable_treatment.exponential_smoothning(data,s,low=False,high=True)
            outlier_treatment+='higher line exponential smoothing'
        except:
            pass

    return outlier_treatment

In [14]:
numerical,categorical = analyze.numerical_categorical_division(df.drop(ignore_cols,axis=1))

In [15]:
categorical

['Sector']

In [16]:
numerical = [x for x in numerical if (x != 'actual_return' and x not in categorical)]

In [17]:
df[numerical].dtypes

High_pct_change                  float64
Low_pct_change                   float64
Last_pct_change                  float64
VWAP_pct_change                  float64
Volume_pct_change                float64
Turnover_pct_change              float64
Trades_pct_change                float64
Deliverable_Volume_pct_change    float64
pct_Deliverble_pct_change        float64
Open_nifty_pct_change            float64
High_nifty_pct_change            float64
Low_nifty_pct_change             float64
Close_nifty_pct_change           float64
Volume_nifty_pct_change          float64
Turnover_nifty_pct_change        float64
Open_auto_pct_change             float64
High_auto_pct_change             float64
Low_auto_pct_change              float64
Close_auto_pct_change            float64
Volume_auto_pct_change           float64
Turnover_auto_pct_change         float64
Open_bank_pct_change             float64
High_bank_pct_change             float64
Low_bank_pct_change              float64
Close_bank_pct_c

In [18]:
edd_df = pd.read_excel('report.xlsx',sheet_name='edd_price_prediction',header=0)

In [19]:
edd_df['outlier_treatment'] = ''
for col in numerical:
    edd_df.loc[edd_df['Var']==col,'outlier_treatment']=automatic_outlier_treatment(df,col)

In [21]:
numerical

['High_pct_change',
 'Low_pct_change',
 'Last_pct_change',
 'VWAP_pct_change',
 'Volume_pct_change',
 'Turnover_pct_change',
 'Trades_pct_change',
 'Deliverable_Volume_pct_change',
 'pct_Deliverble_pct_change',
 'Open_nifty_pct_change',
 'High_nifty_pct_change',
 'Low_nifty_pct_change',
 'Close_nifty_pct_change',
 'Volume_nifty_pct_change',
 'Turnover_nifty_pct_change',
 'Open_auto_pct_change',
 'High_auto_pct_change',
 'Low_auto_pct_change',
 'Close_auto_pct_change',
 'Volume_auto_pct_change',
 'Turnover_auto_pct_change',
 'Open_bank_pct_change',
 'High_bank_pct_change',
 'Low_bank_pct_change',
 'Close_bank_pct_change',
 'Volume_bank_pct_change',
 'Turnover_bank_pct_change',
 'Open_fmcg_pct_change',
 'High_fmcg_pct_change',
 'Low_fmcg_pct_change',
 'Close_fmcg_pct_change',
 'Volume_fmcg_pct_change',
 'Turnover_fmcg_pct_change',
 'Open_it_pct_change',
 'High_it_pct_change',
 'Low_it_pct_change',
 'Close_it_pct_change',
 'Volume_it_pct_change',
 'Turnover_it_pct_change',
 'Open_media_pc

In [20]:
edd_df['outlier_treatment'].value_counts()

flooring at sigma3,Capping at sigma3                111
flooring at p1,capping at p99                        28
flooring at p1,higher line exponential smoothing     11
                                                      2
flooring at sigma2,capping at sigma2                  1
Name: outlier_treatment, dtype: int64

In [22]:
wb = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx',engine='openpyxl')
writer.book = wb
edd_df.to_excel(writer,sheet_name='edd_price_prediction',index=False)
writer.save()
writer.close()

In [23]:
os.chdir('../output_data')

In [24]:
df.to_csv('train_v02.csv',index=False)

In [4]:
df = pd.read_csv('train_v03.csv',header=0)

In [7]:
numerical,categorical = analyze.numerical_categorical_division(df.drop(ignore_cols,axis=1))

In [8]:
for col in numerical:
    df[col] = df[col].apply(lambda x : x if np.abs(x) <= 1 else np.sign(x)*1)

In [8]:
categorical

['Sector']

In [9]:
numerical

['High_pct_change',
 'Low_pct_change',
 'Last_pct_change',
 'VWAP_pct_change',
 'Volume_pct_change',
 'Turnover_pct_change',
 'Trades_pct_change',
 'Deliverable_Volume_pct_change',
 'pct_Deliverble_pct_change',
 'Open_nifty_pct_change',
 'High_nifty_pct_change',
 'Low_nifty_pct_change',
 'Close_nifty_pct_change',
 'Volume_nifty_pct_change',
 'Turnover_nifty_pct_change',
 'Open_auto_pct_change',
 'High_auto_pct_change',
 'Low_auto_pct_change',
 'Close_auto_pct_change',
 'Volume_auto_pct_change',
 'Turnover_auto_pct_change',
 'Open_bank_pct_change',
 'High_bank_pct_change',
 'Low_bank_pct_change',
 'Close_bank_pct_change',
 'Volume_bank_pct_change',
 'Turnover_bank_pct_change',
 'Open_fmcg_pct_change',
 'High_fmcg_pct_change',
 'Low_fmcg_pct_change',
 'Close_fmcg_pct_change',
 'Volume_fmcg_pct_change',
 'Turnover_fmcg_pct_change',
 'Open_it_pct_change',
 'High_it_pct_change',
 'Low_it_pct_change',
 'Close_it_pct_change',
 'Volume_it_pct_change',
 'Turnover_it_pct_change',
 'Open_media_pc

In [10]:
df.to_csv('train_v03.csv',index=False)

# Imputation

In [9]:
conversion_dict={}

In [10]:
numerical

['High_pct_change',
 'Low_pct_change',
 'Last_pct_change',
 'VWAP_pct_change',
 'Volume_pct_change',
 'Turnover_pct_change',
 'Trades_pct_change',
 'Deliverable_Volume_pct_change',
 'pct_Deliverble_pct_change',
 'Open_nifty_pct_change',
 'High_nifty_pct_change',
 'Low_nifty_pct_change',
 'Close_nifty_pct_change',
 'Volume_nifty_pct_change',
 'Turnover_nifty_pct_change',
 'Open_auto_pct_change',
 'High_auto_pct_change',
 'Low_auto_pct_change',
 'Close_auto_pct_change',
 'Volume_auto_pct_change',
 'Turnover_auto_pct_change',
 'Open_bank_pct_change',
 'High_bank_pct_change',
 'Low_bank_pct_change',
 'Close_bank_pct_change',
 'Volume_bank_pct_change',
 'Turnover_bank_pct_change',
 'Open_fmcg_pct_change',
 'High_fmcg_pct_change',
 'Low_fmcg_pct_change',
 'Close_fmcg_pct_change',
 'Volume_fmcg_pct_change',
 'Turnover_fmcg_pct_change',
 'Open_it_pct_change',
 'High_it_pct_change',
 'Low_it_pct_change',
 'Close_it_pct_change',
 'Volume_it_pct_change',
 'Turnover_it_pct_change',
 'Open_media_pc

In [11]:
categorical

['Sector']

In [12]:
for col in categorical:
    conversion_dict[col]=variable_treatment.make_dummies(df,col)

In [13]:
os.chdir('../pickles')

In [14]:
pickle.dump(conversion_dict,open('conversion_dict.pickle','wb'))
pickle.dump(numerical,open('numerical.pickle','wb'))
pickle.dump(categorical,open('categorical.pickle','wb'))

In [15]:
conversion_dict

{'Sector': ['Banking',
  'Automobile',
  'Consumer Goods',
  'Energy - Oil & Gas',
  'Metals',
  'Information Technology',
  'Financial Services',
  'Pharmaceuticals',
  'Cement',
  'Energy - Power',
  'Telecommunication',
  'Construction',
  'Media & Entertainment',
  'Services - Shipping']}

In [16]:
imputation_cols = [col for col in df.columns if col not in numerical+['actual_return']+categorical]

In [17]:
imputation_cols

['Date',
 'Symbol',
 'Series',
 'Prev_Close',
 'Open',
 'Close',
 'Prev_Close_max',
 'Prev_Close_min',
 'Sector_dum_Banking',
 'Sector_dum_Automobile',
 'Sector_dum_Consumer Goods',
 'Sector_dum_Energy - Oil & Gas',
 'Sector_dum_Metals',
 'Sector_dum_Information Technology',
 'Sector_dum_Financial Services',
 'Sector_dum_Pharmaceuticals',
 'Sector_dum_Cement',
 'Sector_dum_Energy - Power',
 'Sector_dum_Telecommunication',
 'Sector_dum_Construction',
 'Sector_dum_Media & Entertainment',
 'Sector_dum_Services - Shipping']

In [33]:
df

,Prev_Close(in 10^5),High_1d(in 10^5),Low_1d(in 10^5),Last_1d(in 10^5),VWAP_1d(in 10^5),Volume_1d(in 10^9),Turnover_1d(in 10^11),Trades_1d(in 10^7),Deliverable_Volume_1d(in 10^9),pct_Deliverble_1d,...,return_2d_500_sqr,return_2d_500_cube,return_3d_500_sqr,return_3d_500_cube,Voltality_500_log,Voltality_log,Sector_dum_0,Sector_dum_1,Sector_dum_2,Sector_dum_3
11278,0.003392,0.003509,0.003365,0.003377,0.003422,0.006868,2350.191491,0.010649,0.004437,0.6460,...,0.001226,0.000043,14.775139,-56.798517,0.365347,-0.071349,0,1,0,1
46936,0.003177,0.003209,0.003136,0.003180,0.003179,0.010770,3423.866024,0.009795,0.007456,0.6922,...,0.001226,0.000043,14.775589,-56.798517,0.365347,0.408431,0,1,0,0
16216,0.028807,0.030342,0.028601,0.028820,0.029379,0.000263,772.463094,0.003012,0.000168,0.6371,...,0.001226,0.000043,14.775813,-56.798517,0.365347,0.459292,0,1,0,0
64226,0.002457,0.002533,0.002450,0.002458,0.002486,0.016326,4057.677992,0.015220,0.005954,0.3647,...,0.001226,0.000043,14.775926,-56.798517,0.365347,0.872609,0,0,0,0
36964,0.008550,0.008685,0.008500,0.008510,0.008594,0.001895,1628.249674,0.004508,0.001408,0.7429,...,0.001226,0.000043,14.775982,-56.798517,0.365347,0.275427,0,1,0,0
72871,0.005073,0.005330,0.005050,0.005065,0.005195,0.002169,1126.928210,0.008639,0.001466,0.6758,...,0.001226,0.000043,14.776010,-56.798517,0.365347,0.855832,1,1,0,0
41749,0.008535,0.008769,0.008485,0.008565,0.008599,0.001090,937.257764,0.003159,0.000526,0.4823,...,0.001226,0.000043,14.776024,-56.798517,0.365347,-0.010814,0,0,0,0
59039,0.002249,0.002366,0.002230,0.002237,0.002304,0.004831,1113.057999,0.007559,0.002999,0.6209,...,0.001226,0.000043,14.776031,-56.798517,0.365347,1.245956,1,0,1,0
50394,0.006273,0.006475,0.006233,0.006242,0.006355,0.001274,809.668119,0.002513,0.000547,0.4290,...,0.001226,0.000043,14.776038,-56.798517,0.365347,0.121770,0,0,0,0
24861,0.002827,0.002894,0.002755,0.002802,0.002845,0.002873,817.266198,0.005269,0.002028,0.7058,...,0.001226,0.000043,14.776038,-56.798517,0.365347,1.473058,1,0,1,0


In [34]:
pickle.dump(imputation_cols,open('imputation_cols.pickle','wb'))

In [35]:
os.chdir('../Imputation_models')

In [37]:
for col in [x for x in numerical if x!='actual_return']:
    data = df[imputation_cols+[col]].dropna(subset=[col])
    x = np.array(data[imputation_cols])
    y = np.array(data[col])
    model = LinearRegression()
    model.fit(x,y)
    del x
    del y
    pickle.dump(model,open(col+'_impute.pickle','wb'))
    if df[col].isnull().any():
        indices = df.loc[df[col].isnull()].index.tolist()
        df.loc[indices,col]=np.array(model.predict(np.array(df.loc[indices,imputation_cols])))
        del indices
    del model
    print(col+' imputed')

Prev_Close(in 10^5) imputed
High_1d(in 10^5) imputed
Low_1d(in 10^5) imputed
Last_1d(in 10^5) imputed
VWAP_1d(in 10^5) imputed
Volume_1d(in 10^9) imputed
Turnover_1d(in 10^11) imputed
Trades_1d(in 10^7) imputed
Deliverable_Volume_1d(in 10^9) imputed
pct_Deliverble_1d imputed
Open_nifty_1d(in 10^5) imputed
High_nifty_1d(in 10^5) imputed
Low_nifty_1d(in 10^5) imputed
Close_nifty_1d(in 10^5) imputed
Volume_nifty_1d(in 10^9) imputed
Turnover_nifty_1d(in 10^11) imputed
Open_auto_1d(in 10^5) imputed
High_auto_1d(in 10^5) imputed
Low_auto_1d(in 10^5) imputed
Close_auto_1d(in 10^5) imputed
Open_bank_1d(in 10^5) imputed
High_bank_1d(in 10^5) imputed
Low_bank_1d(in 10^5) imputed
Close_bank_1d(in 10^5) imputed
Open_fmcg_1d(in 10^5) imputed
High_fmcg_1d(in 10^5) imputed
Low_fmcg_1d(in 10^5) imputed
Close_fmcg_1d(in 10^5) imputed
Open_it_1d(in 10^5) imputed
High_it_1d(in 10^5) imputed
Low_it_1d(in 10^5) imputed
Close_it_1d(in 10^5) imputed
Open_media_1d(in 10^4) imputed
High_media_1d(in 10^4) imput

In [38]:
df[[x for x in numerical if x!='actual_return']].isnull().any()

Prev_Close(in 10^5)               False
High_1d(in 10^5)                  False
Low_1d(in 10^5)                   False
Last_1d(in 10^5)                  False
VWAP_1d(in 10^5)                  False
Volume_1d(in 10^9)                False
Turnover_1d(in 10^11)             False
Trades_1d(in 10^7)                False
Deliverable_Volume_1d(in 10^9)    False
pct_Deliverble_1d                 False
Open_nifty_1d(in 10^5)            False
High_nifty_1d(in 10^5)            False
Low_nifty_1d(in 10^5)             False
Close_nifty_1d(in 10^5)           False
Volume_nifty_1d(in 10^9)          False
Turnover_nifty_1d(in 10^11)       False
Open_auto_1d(in 10^5)             False
High_auto_1d(in 10^5)             False
Low_auto_1d(in 10^5)              False
Close_auto_1d(in 10^5)            False
Open_bank_1d(in 10^5)             False
High_bank_1d(in 10^5)             False
Low_bank_1d(in 10^5)              False
Close_bank_1d(in 10^5)            False
Open_fmcg_1d(in 10^5)             False


In [18]:
default_categorical

NameError: name 'default_categorical' is not defined

In [40]:
df.drop(default_categorical,axis=1,inplace=True)

In [19]:
os.chdir('../output_data')

In [22]:
rename_dict = {col : col.replace(' ','_').replace('-','_').replace('&','_') for col in df.columns if '-' in col or ' ' in col}

In [23]:
rename_dict

{'Sector_dum_Consumer Goods': 'Sector_dum_Consumer_Goods',
 'Sector_dum_Energy - Oil & Gas': 'Sector_dum_Energy___Oil___Gas',
 'Sector_dum_Information Technology': 'Sector_dum_Information_Technology',
 'Sector_dum_Financial Services': 'Sector_dum_Financial_Services',
 'Sector_dum_Energy - Power': 'Sector_dum_Energy___Power',
 'Sector_dum_Media & Entertainment': 'Sector_dum_Media___Entertainment',
 'Sector_dum_Services - Shipping': 'Sector_dum_Services___Shipping'}

In [24]:
df.rename(rename_dict,axis=1,inplace=True)

In [25]:
df.columns

Index(['Date', 'Symbol', 'Series', 'Prev_Close', 'Open', 'High_pct_change',
       'Low_pct_change', 'Last_pct_change', 'Close', 'VWAP_pct_change',
       ...
       'Sector_dum_Metals', 'Sector_dum_Information_Technology',
       'Sector_dum_Financial_Services', 'Sector_dum_Pharmaceuticals',
       'Sector_dum_Cement', 'Sector_dum_Energy___Power',
       'Sector_dum_Telecommunication', 'Sector_dum_Construction',
       'Sector_dum_Media___Entertainment', 'Sector_dum_Services___Shipping'],
      dtype='object', length=175)

In [ ]:
df.to_csv('train_v04.csv',index=False)

In [40]:
df.isnull().any()

Date                                 False
Symbol                               False
Series                               False
Prev_Close                           False
Open                                 False
High_pct_change                      False
Low_pct_change                       False
Last_pct_change                      False
Close                                False
VWAP_pct_change                      False
Volume_pct_change                    False
Turnover_pct_change                  False
Trades_pct_change                    False
Deliverable_Volume_pct_change        False
pct_Deliverble_pct_change            False
Sector                               False
Open_nifty_pct_change                False
High_nifty_pct_change                False
Low_nifty_pct_change                 False
Close_nifty_pct_change               False
Volume_nifty_pct_change              False
Turnover_nifty_pct_change            False
Open_auto_pct_change                 False
High_auto_p